# WSL2를 통한 서버 운영

1. 외부IP:포트 -> 내부IP:포트 -> WSL2가상IP:포트

    - 외부 -> 내부 포트포워딩, 방화벽 해제
    - 내부 -> WSL2 포트포워딩, 방화벽 해제

2. WSL2가상IP:포트 -> web server


-   Web Server -- NGINX
-   Web Application Server (WAS) -- Django
-   Database Server (DB) -- MySQL

Client <-> Web Server <-> WAS <-> DB


## Installation

NGINX - uWSGI - Django : [Setup Guide](https://uwsgi-docs.readthedocs.io/en/latest/tutorials/Django_and_nginx.html)

-   NGINX
    -   apt install nginx
-   uWSGI: NGINX, Django 연결
    -   conda install uwsgi (테스트용)
    -   실 사용을 위해서는 전역 설치가 권장됨. (nginx의 www-data가 가지는 접근권한을 주기 위함인 듯..?)
-   Django
    -   conda install Django


## Django 초기 세팅


In [ ]:
import django

print(django.get_version())

In [ ]:
%cd /home/onsuo/dev/virtual_microscope
! mkdir server_project
! django-admin startproject config server_project

In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! python manage.py -h

In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! python manage.py makemigrations --noinput  # migration 할 data 탐색
! python manage.py migrate  # migration 실행

In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! mkdir apps
%cd apps
! django-admin startapp viewer

In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! python manage.py collectstatic --noinput  # static 리소스 모두 STATIC_ROOT로 모음

## client <-> Django <-> Python

-   기능 개발용으로는 Django 자체 서버를 이용해도 충분함


In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! python manage.py runserver 0.0.0.0:8080

## client <-> uWSGI <-> Python


In [ ]:
# test.py
"""
def application(env, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    return [b'Hello World']
"""

In [ ]:
! uwsgi --http :8400 --wsgi-file test.py

## client <-> uWSGI <-> Django <-> Python


In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! uwsgi --http :8400 --module config.wsgi

## client <-> NGINX <-> socket <-> uWSGI <-> Django <-> Python


In [ ]:
# /etc/nginx/sites-available/mysite_nginx.conf
"""
# the upstream component nginx needs to connect to
upstream django {
    # server unix:///path/to/your/mysite/mysite.sock; # for a file socket
    server 127.0.0.1:8001;
}

# configuration of the server
server {
    # the port your site will be served on
    listen      8000;
    # the domain name it will serve for
    server_name 0.0.0.0;
    charset     utf-8;

    # max upload size
    client_max_body_size 75M;

    # Django media
    location /media/  {
        alias /home/onsuo/dev/virtual_microscope/server_project/media/;
    }

    location /static/ {
        alias /home/onsuo/dev/virtual_microscope/server_project/static/;
    }

    # Finally, send all non-media requests to the Django server.
    location / {
        uwsgi_pass  django;
        include     /home/onsuo/dev/virtual_microscope/server_project/uwsgi_params;
    }
}
"""

# uWSGI 전역 설치 전까지는 nginx.conf의 'user www-data'를 'user <myuser>'로 바꿔두기

In [ ]:
# symbolic-link
! echo 'onsuo' | sudo -S ln -s /etc/nginx/sites-available/virtual_microscope.conf /etc/nginx/sites-enabled/

In [ ]:
! echo 'onsuo' | sudo -S systemctl restart nginx

In [ ]:
%cd /home/onsuo/dev/virtual_microscope/server_project
! uwsgi --socket :8001 --module config.wsgi  # using port

In [ ]:
! echo 'onsuo' | sudo -S systemctl stop nginx